# Exploratory Data Analysis and Building ML Model for Diabetes Prediction

**This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.**

**The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.**

Id: Unique identifier for each data entry.

Pregnancies: Number of times pregnant.

Glucose: Plasma glucose concentration over 2 hours in an oral glucose tolerance test.

BloodPressure: Diastolic blood pressure (mm Hg).

SkinThickness: Triceps skinfold thickness (mm).

Insulin: 2-Hour serum insulin (mu U/ml).

BMI: Body mass index (weight in kg / height in m^2).

DiabetesPedigreeFunction: Diabetes pedigree function, a genetic score of diabetes.

Age: Age in years.

Outcome: Binary classification indicating the presence (1) or absence (0) of diabetes.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('diabetes.csv')

In [ ]:
df.head()

In [ ]:
print("Number of rows present in the dataset are: ", df.shape)

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
import seaborn as sns

In [ ]:
df['Outcome'].value_counts()

In [ ]:
fig, ax = plt.subplots()

labels = ['Diabetic', 
         'Non-Diabetic']
percentages = [34.89, 65.10]
explode=(0.1,0)
ax.pie(percentages, explode=explode, labels=labels, autopct='%1.0f%%', 
       shadow=False, startangle=0,   
       pctdistance=1.2,labeldistance=1.4)
ax.legend(frameon=False, bbox_to_anchor=(1.5,0.8))
plt.show()

**The distribution of the dependent variable is not skewed or imbalanced. We can move ahead with the same data without having to apply SMOTE or undersampling or oversampling techniques. But we do need to make that we distribution of the classes remain same when we split our data to train and test set.**

**Before we move ahead, we need to check what are the minimum values for each column, certain columns like Glucose or Insulin can not have values as 0. Therefore, we need to take care of such values.**

In [ ]:
for col in df.columns:
    print("The minimum value fore the columns {} is {}".format(col, df[col].min()))

**Now out of the above columns having zero as their minima, only Pregnancie Column can take the values as zero, so what should do we do with those columns that have zero as their minimum even if they aren't supposed to?**

# Null Values

In [ ]:
def msv_1(data, thresh = 20, color = 'black', edgecolor = 'black', height = 3, width = 15):
    
    plt.figure(figsize = (width, height))
    percentage = (data.isnull().mean()) * 100
    percentage.sort_values(ascending = False).plot.bar(color = color, edgecolor = edgecolor)
    plt.axhline(y = thresh, color = 'r', linestyle = '-')
    
    plt.title('Missing values percentage per column', fontsize=20, weight='bold' )
    
    plt.text(len(data.isnull().sum()/len(data))/1.7, thresh+2.5, f'Columns with more than {thresh}% missing values', fontsize=12, color='crimson',
         ha='left' ,va='top')
    plt.text(len(data.isnull().sum()/len(data))/1.7, thresh - 0.5, f'Columns with less than {thresh}% missing values', fontsize=12, color='green',
         ha='left' ,va='top')
    plt.xlabel('Columns', size=15, weight='bold')
    plt.ylabel('Missing values percentage')
    plt.yticks(weight ='bold')
    
    return plt.show()
msv_1(df, 20, color=sns.color_palette('Reds',15))

**You might be wondering that there are no null values in the dataset, but are you sure? Remember what we had discussed in the previous section where certain columns were having zero as their minima eventhough they aren't supposed to. Those values will be considered as null values. Let's replace the zeros present in the Glucose, BloodPressure, SkinThickness, Insulin, and BMI columns with null.**

In [ ]:
df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.nan)

In [ ]:
msv_1(df, 20, color=sns.color_palette('Reds',15))

In [ ]:
# Checking for NULLs in the data
df.isnull().sum()


**We can observe that Insulin column has close to 50% zero or null values, followed by SkinThickness that has close to 30% missing values. We will be filling these values later.**

# Exploratory Data Analysis

**In this section, we will be doing some basic Exploratory Data Analysis to get the "feel" of the data, we will be checking the distributions, the correlations etc of the different columns and try to remove the null values present.**

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(15, 10))
axes = axes.flatten()
ax_idx = 0
columns = df.drop('Outcome', axis = 1).columns
for col in columns:
    df[col].plot(kind = 'hist', ax = axes[ax_idx], title = col, color = next(color_cycle))
    ax_idx += 1

plt.suptitle('Sales Trend according to Departments')
plt.tight_layout()
plt.show()

**Let's check the skewness of each of the columns.**

**Skewness refers to the amount of asymmetry in the given feature or in other words amount of distortions from the normal distribution. The peak of the histogram represents the mode.**

In [ ]:
from scipy.stats import skew
for col in df.drop('Outcome', axis = 1).columns:
    print("Skewness for the column {} is {}".format(col, df[col].skew()))

**Columns like Pregnancies, Glucose, BloodPressure, SkinThickness and BMI are not that much skewed. We can fill null values with the mean for these columns, but for columns like Insulin and DiabetesPedigreeFunction, we will have to replace them will median due to the effect of skewness.**

In [ ]:
from sklearn.impute import SimpleImputer
df_cut=df.copy()
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df_cut[['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']])
df_cut[['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']] = imputer.transform(df[['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']])

In [ ]:
df['Insulin'] = df['Insulin'].fillna(df['Insulin'].median()) # Filling null values with the median.

for col in ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']:
    df[col] = df[col].fillna(df[col].mean())

In [ ]:
msv_1(df, 10, color=sns.color_palette('Greens',15))

In [ ]:
df.isnull().sum()

In [ ]:
df_cut,df

**All null values are taken care of now**

In [ ]:
def mean_target(var):
    """
    A function that will return the mean values for 'var' column depending on whether the person
    is diabetic or not
    """
    return pd.DataFrame(df.groupby('Outcome').mean()[var])

In [ ]:
def distplot(col_name):
    """
    A function that will plot the distribution of column 'col_name' for diabetic and non-diabetic people separately
    """
    plt.figure()
    sns.histplot(df[col_name][df.Outcome == 1], color ="red",kde=True)
    sns.histplot(df[col_name][df.Outcome == 0], color ="lightblue",kde=True)
    plt.legend(['Diabetes', 'No Diabetes'])

## Pregnancies

In [ ]:
distplot('Pregnancies')

In [ ]:
mean_target('Pregnancies')

**We can see that the number of pregnancies is high for the diabetic people**

## Insulin

In [ ]:
distplot('Insulin')

In [ ]:
mean_target('Insulin')

**Diabetic People tend to have more Insulin level.**

## BloodPressure

In [ ]:
distplot('BloodPressure')


In [ ]:
mean_target('BloodPressure')

**The mean of the blood pressure is greater for diabetic people as compared to the non-diabetic people**

## Glucose

In [ ]:
distplot('Glucose')

In [ ]:
mean_target('Glucose')

**Diabetic People tend to have much higher Glucose level**

# Comman Man Analysis

**Let's think like a common man, and analyze the data.**

**First, we would know what is the effect of Age on the Outcome because we have heard that as the age increases, the chances of diabetes also commonly increases.**

In [ ]:
sns.boxplot(x = 'Outcome', y = 'Age', data = df)
plt.title('Age vs Outcome')
plt.show()

**Yes, we were right, the median of the age of diabetic people is greater than that of non-diabetic people.**

**Let's also check the effect of Blood Pressure on the Outcome.**

In [ ]:
sns.boxplot(x = 'Outcome', y = 'BloodPressure', data = df)
plt.title('BP vs Outcome')
plt.show()

**The median of the BloodPressure of diabetic people lies close to the 75th Percentile of non-diabetic people.**

**The next thing a common man would check is the relationship between age and BP**

In [ ]:
sns.jointplot(x='Age',y='BloodPressure', data=df, kind = 'reg', color = 'green')

**Hmm, as the age increases, generally the Blood Pressure also increases**

**One would also want to know the chances of getting diabetes, if it is common in the family. We can check that with the Diabetes Pedigree Function.**

In [ ]:
my_pal = {0: "lightgreen", 1: "lightblue"}
sns.boxplot(x = 'Outcome', y = 'DiabetesPedigreeFunction', data = df)
plt.title('DPF vs Outcome')
plt.show()

**Quite a proportion of people having high DPF does not end up having Diabetes.  But usually the diabetic people have DPF value close to 0.5 (50th Percentile)**

## Gluscose Level

In [ ]:
my_pal = {0: "lightgrey", 1: "lightyellow"}
sns.boxplot(x = 'Outcome', y = 'Glucose', data = df)
plt.title('Glucose vs Outcome')
plt.show()

**Wow! the median of the Glucose level of Diabetic People is greater than the 75th Percentile of the glucose level of non-diabetic people. Therefore having a high glucose level does increase the chances of having diabetes.**

## Insulin

**Let's first check whether there is any relation between glucose and insulin level.**

In [ ]:
sns.jointplot(x='Insulin',y='Glucose', data=df, kind = 'reg', color = 'red')
plt.show()

**We can see that as the insulin level increases, the Glucose level also increases.**

In [ ]:
sns.boxplot(x = 'Outcome', y = 'Insulin', data = df)
plt.title('Insulin vs Outcome')
plt.show()

## Body Mass Index

**Body mass index (BMI) is a measure of body fat based on height and weight that applies to adult men and women. Does having a higher BMI leads to more chances of being diabetic? Let's check that out!**

In [ ]:
sns.boxplot(x = 'Outcome', y = 'BMI', data = df)
plt.title('BMI vs Outcome')
plt.show()

**Indeed, the Median BMI of the Diabetic People is greater than the Median BMI of the Non-Diabetic people.**

# Correlation Matrix

In [ ]:
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1.0, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot = True)

**From the above heatmap, we can observe that all the features are weakly correlated, so that removes multicollinearity out of equation. Multicollinearity (also collinearity) is a phenomenon in which one predictor variable in a multiple regression model can be linearly predicted from the others with a substantial degree of accuracy. Models like Logistic Regression assumes the presence of non-collinearity among the features, if multicollinearity is present it can lead to the bad performance of such models.**

In [ ]:
from scipy.stats import zscore

outlBmi = list(np.where(np.abs(np.abs(zscore(df['BMI']))) > 3)[0])
print("BMI Outliers: ", outlBmi)
print("Total outlier BMI: ", len(outlBmi), "\n")


outlInsulin = list(np.where(np.abs(np.abs(zscore(df['Insulin']))) > 3)[0])
print("Insulin Outliers: ", outlInsulin)
print("Total outlier Insulin: ", len(outlInsulin), "\n")

outl = list(set(outlBmi + outlInsulin))


In [ ]:
df.iloc[outlInsulin, :]

In [ ]:
sns.boxplot(x = 'Outcome', y = 'BMI', data = df)
plt.title('BMI vs Outcome')
plt.show()

df_outlier=df.copy()
df_outlier.drop(df_outlier.index[list(outlInsulin)], inplace=True)

sns.boxplot(x = 'Outcome', y = 'BMI', data = df_outlier)
plt.title('BMI vs Outcome')
plt.show()

# Dataset Splitting and Features Scaling

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Outcome', axis = 1)

y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42, stratify = y)

**Standardizing a dataset involves rescaling the distribution of values so that the mean of observed values is 0 and the standard deviation is 1.**

**This can be thought of as subtracting the mean value or centering the data. Scaling the features is of utmost importance because different features are in different scales. Let's say the Age has values in double digits, whereas the DPF is of the kind float, the effect of the Age feature will be more as compared to the DPF**

**Best practice is to use only the training set to figure out how to scale / normalize, then blindly apply the same transform to the test set.**

**For example, say you're going to normalize the data by removing the mean and dividing out the variance. If you use the whole dataset to figure out the feature mean and variance, you're using knowledge about the distribution of the test set to set the scale of the training set - 'leaking' information.**

**The right way to do this is to use only the training set to calculate the mean and variance, normalize the training set, and then at test time, use that same (training) mean and variance to normalize the test set.**

#### The Big Question – Normalize or Standardize?

Normalization vs. standardization is an eternal question among machine learning newcomers. Let me elaborate on the answer in this section.

Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. This can be useful in algorithms that do not assume any distribution of the data like K-Nearest Neighbors and Neural Networks.

Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution. However, this does not have to be necessarily true. Also, unlike normalization, standardization does not have a bounding range. So, even if you have outliers in your data, they will not be affected by standardization.

However, at the end of the day, the choice of using normalization or standardization will depend on your problem and the machine learning algorithm you are using. There is no hard and fast rule to tell you when to normalize or standardize your data.

Robust Scaler
When working with outliers we can use Robust Scaling for scakling our data, It scales features using statistics that are robust to outliers. This method removes the median and scales the data in the range between 1st quartile and 3rd quartile. i.e., in between 25th quantile and 75th quantile range. This range is also called an Interquartile range. The median and the interquartile range are then stored so that it could be used upon future data using the transform method. If outliers are present in the dataset, then the median and the interquartile range provide better results and outperform the sample mean and variance. RobustScaler uses the interquartile range so that it is robust to outliers

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train =  pd.DataFrame(sc.fit_transform(X_train),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age'])
X_test = pd.DataFrame(sc.fit_transform(X_test),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age'])

# Baseline Models

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

def evaluation(model, x_train_std, y_train, x_test, y_test, train = True):
    """
    A function that returns the score of every evaluation metrics
    """
    if train == True:
        pred = model.predict(x_train_std)
        classifier_report = pd.DataFrame(classification_report(y_train, pred, output_dict = True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"F1 Score: {round(f1_score(y_train, pred), 2)}")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{classifier_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    if train == False:
        pred = model.predict(x_test)
        classifier_report = pd.DataFrame(classification_report(y_test, pred, output_dict = True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"F1 Score: {round(f1_score(y_test, pred), 2)}")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{classifier_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver = 'liblinear')
lr.fit(X_train, y_train)

evaluation(lr, X_train, y_train, X_test, y_test, True)
print()
evaluation(lr, X_train, y_train, X_test, y_test, False)

In [ ]:
train_score_lr = round(accuracy_score(y_train, lr.predict(X_train)) * 100, 2)
test_score_lr = round(accuracy_score(y_test, lr.predict(X_test)) * 100, 2)

In [ ]:
coeff = list(lr.coef_[0])
labels = list(df.columns[:-1])
features = pd.DataFrame()
features['Features'] = labels
features['importance'] = coeff
features.sort_values(by=['importance'], ascending=True, inplace=True)
features['positive'] = features['importance'] > 0
features.set_index('Features', inplace=True)
features.importance.plot(kind='barh', figsize=(11, 6),color = features.positive.map({True: 'blue', False: 'red'}))
plt.xlabel('Importance')


In [ ]:
models = {
           'Train Accuracy': [train_score_lr],
          'Test Accuracy' : [test_score_lr]
         }

models = pd.DataFrame(models, index = ['Logistic Regression'])
models.head()

# Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier

kfold = StratifiedKFold(n_splits=10)

random_state = 0
lr=LogisticRegression(C=1,random_state = random_state)

cv_results=(cross_val_score(lr, X_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=-1))

cv_results

In [ ]:
np.mean(cv_results),np.std(cv_results)

# Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFECV
estimator = LogisticRegression(random_state=random_state)
rfecv = RFECV(estimator=estimator, cv=StratifiedKFold(10, random_state=random_state, shuffle=True), scoring="accuracy")
rfecv.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(rfecv.cv_results_["mean_test_score"]) + 1), rfecv.cv_results_["mean_test_score"], color='#303F9F', linewidth=3)
plt.grid()
plt.xticks(range(1, X.shape[1]+1))
plt.xlabel("Number of Selected Features")
plt.ylabel("CV Score")
plt.title("Recursive Feature Elimination (RFE)")
plt.show()

print("The optimal number of features: {}".format(rfecv.n_features_))

In [ ]:
X_rfe = X.iloc[:, rfecv.support_]

In [ ]:
print("\"X\" dimension: {}".format(X.shape))
print("\"X\" column list:", X.columns.tolist())
print("\"X_rfe\" dimension: {}".format(X_rfe.shape))
print("\"X_rfe\" column list:", X_rfe.columns.tolist())


In [ ]:
X_train, X_test, X_rfe_train, X_rfe_test, y_train, y_test = train_test_split(X, X_rfe, y, 
                                                                             train_size=0.8, 
                                                                             stratify=y,
                                                                             random_state=random_state)


In [ ]:
# Original dataset
print("Model training using original data: started!")
lr=LogisticRegression()
lr.fit(X_train, y_train)
print("Model training using original data: done!\n")
# Feature-selected dataset
print("Model training using feature-selected data: started!")
rfecv = RFECV(estimator=estimator, cv=StratifiedKFold(10, random_state=random_state, shuffle=True), scoring="accuracy")
rfecv.fit(X_rfe_train, y_train)
print("Model training using feature-selected data: done!")


In [ ]:
# Original dataset
acc = []
y_pred = lr.predict(X_test)
acc.append(accuracy_score(y_test, y_pred))

# Feature selected dataset
acc_rfe = []
y_rfe_pred = rfecv.predict(X_rfe_test)
acc_rfe.append(accuracy_score(y_test, y_rfe_pred))
    
acc_all = pd.DataFrame({"Original dataset": acc, "Feature-selected dataset": acc_rfe})
acc_all

In [ ]:
# Metrics
rfecv.fit(X_rfe_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_curve
# calculating the probabilities
y_rfe_pred  = rfecv.predict_proba(X_rfe_test)[:,1]
y_pred  = lr.predict_proba(X_test)[:,1]


# instantiating the roc_cruve
fpr,tpr,threshols=roc_curve(y_test,y_rfe_pred )

fpr1,tpr1,threshols1=roc_curve(y_test,y_pred)


# plotting the curve
plt.plot([0,1],[0,1],"k--",'r+')
plt.plot(fpr,tpr,'b',label='Logistic Regression')
plt.plot(fpr1,tpr1,'r',label='Logistic Regression all')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Logistric Regression ROC Curve")
plt.legend()
plt.show()

### Interpreting ROC Plot:

Interpreting the ROC plot is very different from a regular line plot. Because, though there is an X and a Y-axis, we don't read it as: for an X value of 0.25, the Y value is .9.

Instead, what we have here is a line that traces the probability cutoff from 1 at the bottom-left to 0 in the top right.

This is a way of analyzing how the sensitivity and specificity perform for the full range of probability cutoffs, that is from 0 to 1.

Ideally, if we have a perfect model, all the events will have a probability score of 1 and all non-events will have a score of 0. For such a model, the area under the ROC will be a perfect 1.

So, if we trace the curve from bottom left, the value of probability cutoff decreases from 1 towards 0. If we have a good model, more of the real events should be predicted as events, resulting in high sensitivity and low FPR. In that case, the curve will rise steeply covering a large area before reaching the top-right.

Therefore, the larger the area under the ROC curve, the better is the model.

The ROC curve is the only metric that measures how well the model does for different values of prediction probability cutoffs.

In [ ]:
roc_auc_logreg = cross_val_score(rfecv, X_rfe_train, y_train, cv = kfold, scoring = 'roc_auc').mean()
roc_auc_logreg_all = cross_val_score(lr, X_train, y_train, cv = kfold, scoring = 'roc_auc').mean()

In [ ]:
roc_auc_logreg,roc_auc_logreg_all

In [ ]:
from sklearn import metrics

logreg_matrix = metrics.confusion_matrix(y_test, np.round(y_rfe_pred,0))
print(logreg_matrix)


In [ ]:
logreg_matrix = metrics.confusion_matrix(y_test, np.round(y_pred,0))
print(logreg_matrix)